In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="XccuiW0r9cso9fh8Kg4n")
project = rf.workspace("deepl-n1zpl").project("cut_the_bubble-2")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cut_the_bubble-2-1 in yolov8:: 100%|██████████| 2204/2204 [00:00<00:00, 3482.48it/s]


In [7]:
# Import thư viện cần thiết
from ultralytics import YOLO
import torch
import os

# Kiểm tra GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Đường dẫn đến dataset
dataset_path = "./cut_the_bubble-2-1"
data_yaml_path = os.path.join(dataset_path, "data.yaml")

# Kiểm tra file data.yaml
if not os.path.exists(data_yaml_path):
    print(f"Không tìm thấy file data.yaml tại: {data_yaml_path}")
    data_yaml_path = input("Nhập đường dẫn đến file data.yaml: ")

# Khởi tạo mô hình YOLOv8 tiêu chuẩn (không phải OBB)
model_size = 's'  # 'n' (nano), 's' (small), 'm' (medium), 'l' (large), 'x' (xlarge)
model_path = f'yolov8{model_size}.pt'  # Đã bỏ "-obb" để sử dụng mô hình tiêu chuẩn
model = YOLO(model_path)

# Hiển thị thông số huấn luyện
print(f"\n=== THÔNG TIN HUẤN LUYỆN ===")
print(f"- Model: YOLOv8{model_size} (Standard Detection)")
print(f"- Dataset: {data_yaml_path}")
print(f"- Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Huấn luyện mô hình
results = model.train(
    data=data_yaml_path,     # Đường dẫn đến file cấu hình dataset
    epochs=200,              # Số epochs
    imgsz=640,               # Kích thước ảnh đầu vào
    batch=16,                # Kích thước batch - giảm xuống nếu hết bộ nhớ GPU
    device=0 if torch.cuda.is_available() else 'cpu',  # Sử dụng GPU nếu có
    patience=30,             # Early stopping
    project='cut_bubble',    # Tên project
    name='model_v3',         # Đổi tên để tránh ghi đè mô hình OBB
    exist_ok=True            # Ghi đè nếu thư mục đã tồn tại
)

# In ra đường dẫn đến model tốt nhất
print(f"\nBest model saved at: {results.best}")

# Đánh giá mô hình
metrics = model.val()
print(f"\n=== KẾT QUẢ ĐÁNH GIÁ ===")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")

# Test mô hình (thay thế đường dẫn bằng đường dẫn tới ảnh test của bạn)
print("\n=== ĐANG TEST MÔ HÌNH ===")
test_image = os.path.join(dataset_path, "test", "images", "test_0.jpg")  # Thay đổi nếu cần
if os.path.exists(test_image):
    results = model.predict(test_image, conf=0.25, save=True)
    print(f"Đã lưu kết quả test tại: {results[0].save_dir}")
else:
    print(f"Không tìm thấy ảnh test tại: {test_image}")
    print("Bạn có thể test mô hình sau với lệnh:")
    print(f"    model = YOLO('{results.best}')")
    print("    results = model.predict('đường_dẫn_ảnh_test', conf=0.25, save=True)")

GPU available: True
GPU device: NVIDIA GeForce RTX 4070

=== THÔNG TIN HUẤN LUYỆN ===
- Model: YOLOv8s (Standard Detection)
- Dataset: ./cut_the_bubble-2-1\data.yaml
- Device: GPU
Ultralytics 8.3.148  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./cut_the_bubble-2-1\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.9

train: Scanning E:\yolo\cut_the_bubble-2-1\train\labels... 792 images, 8 backgrounds, 0 corrupt: 100%|██████████| 792/792 [00:01<00:00, 606.53it/s]

train: New cache created: E:\yolo\cut_the_bubble-2-1\train\labels.cache


val: Fast image access  (ping: 0.10.0 ms, read: 15.69.8 MB/s, size: 129.0 KB)


val: Scanning E:\yolo\cut_the_bubble-2-1\valid\labels... 203 images, 1 backgrounds, 0 corrupt: 100%|██████████| 203/203 [00:00<00:00, 511.76it/s]

...

                   all        203        897      0.935      0.915      0.949      0.824



200 epochs completed in 0.467 hours.
Optimizer stripped from cut_bubble\model_v3\weights\last.pt, 22.5MB
Optimizer stripped from cut_bubble\model_v3\weights\best.pt, 22.5MB

Validating cut_bubble\model_v3\weights\best.pt...
Ultralytics 8.3.148  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  3.77it/s]


                   all        203        897      0.947      0.903      0.953      0.829
------------------------------         88        259      0.903      0.892      0.937      0.813
This dataset was exported via roboflow.com on July 5- 2024 at 9-13 AM GMT         24         36      0.964      0.861      0.957       0.86
             back_text         36         94      0.954      0.936      0.981      0.868
                bubble        103        421       0.99      0.976      0.993      0.932
bubble_cutter - v5 2024-07-05 9-08am         32         87      0.922      0.851        0.9      0.673
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to cut_bubble\model_v3


AttributeError: 'DetMetrics' object has no attribute 'best'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (Dict[int, str]): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (Dict[str, float]): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
    

In [ ]:
from ultralytics import YOLO

# Tải mô hình đã huấn luyện (best.pt)
model = YOLO('cut_bubble/model_v1/weights/best.pt')

# Đường dẫn đến file data.yaml
data_yaml_path = "./cut_the_bubble-2-1/data.yaml"

# Tiếp tục huấn luyện với điều chỉnh các tham số
results = model.train(
    data=data_yaml_path,
    epochs=50,              # Tổng số epochs 
    imgsz=640,               # Kích thước ảnh đầu vào
    batch=16,                # Kích thước batch
    patience=30,            
    project='cut_bubble',    # Giữ nguyên tên project
    name='model_v2',         # Đổi tên để không ghi đè
    exist_ok=True,           # Cho phép ghi đè thư mục nếu tồn tại
    device=0,                # Sử dụng GPU
    resume=False             # Không sử dụng resume=True, mà bắt đầu huấn luyện mới từ weights hiện tại
)

# Sau khi huấn luyện xong, đánh giá mô hình
metrics = model.val()
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")

Ultralytics 8.3.148  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./cut_the_bubble-2-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=cut_bubble/model_v1/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model_v2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective

train: Scanning E:\yolo\cut_the_bubble-2-1\train\labels.cache... 792 images, 8 backgrounds, 0 corrupt: 100%|██████████| 792/792 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.30.0 ms, read: 11.47.2 MB/s, size: 129.0 KB)


val: Scanning E:\yolo\cut_the_bubble-2-1\valid\labels.cache... 203 images, 1 backgrounds, 0 corrupt: 100%|██████████| 203/203 [00:00<?, ?it/s]


Plotting labels to cut_bubble\model_v2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to cut_bubble\model_v2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 0.130 hours.
Optimizer stripped from cut_bubble\model_v2\weights\last.pt, 23.3MB
Optimizer stripped from cut_bubble\model_v2\weights\best.pt, 23.3MB

Validating cut_bubble\model_v2\weights\best.pt...
Ultralytics 8.3.148  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv8s-obb summary (fused): 81 layers, 11,413,506 parameters, 0 gradients, 29.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:04<00:00,  3.11it/s]


                   all        203        897      0.944      0.866      0.924      0.867
------------------------------         88        259       0.93      0.824      0.901      0.845
This dataset was exported via roboflow.com on July 5- 2024 at 9-13 AM GMT         24         36      0.925      0.861      0.846      0.816
             back_text         36         94      0.964      0.864      0.972      0.931
                bubble        103        421      0.995      0.955      0.989      0.967
bubble_cutter - v5 2024-07-05 9-08am         32         87      0.904      0.828       0.91      0.778
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to cut_bubble\model_v2
Ultralytics 8.3.148  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv8s-obb summary (fused): 81 layers, 11,413,506 parameters, 0 gradients, 29.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1741.3367.1 MB/s, size: 123.7 KB)


val: Scanning E:\yolo\cut_the_bubble-2-1\valid\labels.cache... 203 images, 1 backgrounds, 0 corrupt: 100%|██████████| 203/203 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:04<00:00,  2.93it/s]


                   all        203        897      0.944      0.866      0.923      0.868
------------------------------         88        259       0.93      0.824        0.9      0.845
This dataset was exported via roboflow.com on July 5- 2024 at 9-13 AM GMT         24         36      0.924      0.861      0.846      0.816
             back_text         36         94      0.964      0.864      0.972      0.931
                bubble        103        421      0.995      0.955      0.989      0.967
bubble_cutter - v5 2024-07-05 9-08am         32         87      0.904      0.828       0.91      0.778
Speed: 0.8ms preprocess, 3.6ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to cut_bubble\model_v2
mAP50-95: 0.8675
mAP50: 0.9234
